# Test Model

In [1]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.getcwd()).parent))

from settings.global_settings import GlobalSettings

config = GlobalSettings.get_config(
    config_file = "../config.ini",
    secrets_file = "../secrets.ini"
)
from dataset.video_loader import VideoDataLoader
from dataset.video_dataset import VideoDataset
from model.training_loop import train
from torch.utils.data import random_split
from model.multimodal_har_model import MultiModalHARModel

2025-10-19 20:07:48,619 - INFO - Sentry DSN set to: https://f4f21cc936b3ba9f5dbc1464b7a40ea4@o4504168838070272.ingest.us.sentry.io/4506464560414720
2025-10-19 20:07:48,620 - INFO - Sentry initialized with environment: development


Loading config...
Loading secrets...


## Initializing Training

**Creating Dataloaders**

In [2]:
video_data_loader = VideoDataLoader(
    path=config.model_settings.video_data_dir
)

video_dataset = VideoDataset(
    video_data_loader=video_data_loader,
)

len(video_dataset)
for _ in video_dataset:
    pass
len(video_dataset.labels_map)

2025-10-19 20:07:50,588 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-10-19 20:07:50,930 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-10-19 20:07:51,282 - INFO - [VideoDataLoader] Loding action videos for action: a03
2025-10-19 20:07:51,527 - INFO - [VideoDataLoader] Loding action videos for action: a04
2025-10-19 20:07:51,950 - INFO - [VideoDataLoader] Loding action videos for action: a05
2025-10-19 20:07:52,195 - INFO - [VideoDataLoader] Loding action videos for action: a06
2025-10-19 20:07:52,535 - INFO - [VideoDataLoader] Loding action videos for action: a08
2025-10-19 20:07:53,198 - INFO - [VideoDataLoader] Loding action videos for action: a09
2025-10-19 20:07:53,756 - INFO - [VideoDataLoader] Loding action videos for action: a11
2025-10-19 20:07:53,994 - INFO - [VideoDataLoader] Loding action videos for action: a12


10

**Splitting Train and Test Datasets**

In [3]:
num_total = len(video_dataset)
num_train = int(0.8 * num_total)
num_test = num_total - num_train
train_dataset, test_dataset = random_split(video_dataset, [num_train, num_test])

**Creating Model**

In [4]:

har_model = MultiModalHARModel(
    obj_in=train_dataset[0].graphs_objects[0].x.shape[1],
    joint_in=train_dataset[0].graphs_joints[0].x.shape[1],
    gat_hidden=128,
    gat_out=128,
    temporal_hidden=128,
    num_classes=len(video_dataset.labels_map), 
)

## Training Model

In [5]:
train_history = train(
    model=har_model,
    video_dataset=train_dataset,
    device='mps',
    epochs=1,
)

2025-10-19 20:07:58,989 - INFO - Starting training loop...
Epoch 1/1: 100%|██████████| 1180/1180 [06:10<00:00,  3.18it/s]
2025-10-19 20:14:09,639 - INFO - Epoch 1/1, Loss: 2.3120


**Saving Model**

In [6]:
har_model.save(
    training_history=train_history
)

2025-10-19 20:14:09,821 - INFO - Saving model to /Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v0.0.0_nw_ucla_2025-10-19 20:14:09.821734.pht...
2025-10-19 20:14:09,864 - INFO - Model saved successfully.
